In [1]:
import pandas as pd
import numpy as np
import pickle
import gymnasium as gym
from gymnasium.spaces import Box, Discrete
import statistics
from utils import methods
from utils.rl_environments import GammaIntervalsEnv1

with open(methods.file_path('prof_data/data1_intervals.pkl', 'data'), 'rb') as f:
    intervals = pickle.load(f)

df = pd.read_csv(methods.file_path('prof_data/data1.csv', 'data'))

df['intervals'] = intervals
display(df.head())



,total,travel_time,h,c,intervals
0,43.0,3.535281,5.320843,1.787906,"[0.1306593104459917, 0.08214531495921755, 0.02..."
1,48.0,41.459338,0.120220,1.670595,"[4.276706517433805, 3.2544721073860834, 3.7400..."
2,31.0,114.792988,0.146738,6.669483,"[16.079087401217105, 11.608519241181346, 14.23..."
3,52.0,976.197274,0.003427,0.570308,"[28.206915811705233, 6.758341759637238, 24.521..."
4,39.0,41.909987,0.105977,3.407713,"[0.9262261308152662, 1.9254123526889755, 3.236..."


In [2]:
from stable_baselines3 import PPO
model = PPO.load("ppo")

env = GammaIntervalsEnv1()
rewards = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    rewards[i] = total_reward
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")


df['rewards'] = rewards

df.head()

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'h': -1, 'c': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'alpha_hat': -1, 'beta_hat': -1, 'u_star_hat': -1, 'last_update': -1}}
Episode 0 reward: -0.15929345061947325 | cur_time = 0.33871069599653525
Episode 1 reward: -16.885335829074688 | cur_time = 11.471261353239978
Episode 2 reward: -41.999261293478334 | cur_time = 42.32336413816172
Episode 3 reward: -0.37602287216250907 | cur_time = 60.78652512371029
Episode 4 reward: -8.188633779199423 | cur_time = 6.4883595532524385
Episode 5 reward: -100.68872609373456 | cur_time = 2859.6977671615878
Episode 6 reward: -24.332400284277508 | cur_time = 0.6717901002504495
Episode 7 reward: -2.882353667336145 | cur_time = 21.837525017502852
Episode 8 reward: -11.673091891566026 | cur_time = 18.872576122540742
Episode 9 reward: -431.83611238636115 | cur_time = 731.1480170021765
Episode 10 reward: -10.887159286968732 | cur

,total,travel_time,h,c,intervals,rewards
0,43.0,3.535281,5.320843,1.787906,"[0.1306593104459917, 0.08214531495921755, 0.02...",-0.159293
1,48.0,41.459338,0.120220,1.670595,"[4.276706517433805, 3.2544721073860834, 3.7400...",-16.885336
2,31.0,114.792988,0.146738,6.669483,"[16.079087401217105, 11.608519241181346, 14.23...",-41.999261
3,52.0,976.197274,0.003427,0.570308,"[28.206915811705233, 6.758341759637238, 24.521...",-0.376023
4,39.0,41.909987,0.105977,3.407713,"[0.9262261308152662, 1.9254123526889755, 3.236...",-8.188634


In [3]:
import plotly.graph_objects as go

# Calculate means
means = df[['rewards', 'h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Rewards, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[['rewards', 'h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Rewards, H, and C", yaxis_title="Median Value")
fig.show()







